In [13]:
import numpy as np
from collections import namedtuple
from ortools.sat.python import cp_model
import timeit
import random

In [14]:
''' Algorithmic Approach : use solve_schedule_algo(intervals, num_resources) '''

# Proof: http://www.cs.toronto.edu/~milad/csc373/lectures/T1.pdf
# O(n^2)
def solve_schedule_algo(intervals, num_resources):
  sorted_end   = sorted(intervals, key=lambda x: x[1])    # n*log(n)
  schedules = [[] for _ in range(num_resources)]

  for interval in sorted_end: 
    # get schedule with smallest end time such that the start time of interval is greater than endtime
    index = assign_schedule(interval[0], schedules)
    if index > -1: schedules[index].append(interval)

  return sum([len(schedule) for schedule in schedules]), schedules  

def assign_schedule(start_time, schedules):
  index = -1
  latest_end = -1
  for i, schedule in enumerate(schedules): 
    schedule_end_time = 0 if len(schedule) < 1 else schedule[-1][1]
    if schedule_end_time > start_time:
      continue
    elif schedule_end_time > latest_end:
      latest_end = schedule_end_time
      index = i
  return index

In [15]:
''' SMT Approach : use solve_schedule_smt(intervals, num_resources) '''

def solve_schedule_smt(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Maximize Length of Schedule
  model.Maximize(sum(assign_resource.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return solver.ObjectiveValue(), solution

In [16]:
''' SMT Approach + Resource Optimization : use solve_schedule_opt(intervals, num_resources) '''

def solve_schedule_opt(intervals, num_resources):
  # Solve Intermediate Solution
  obj = int(solve_schedule_smt(intervals, num_resources)[0])

  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Add Optimal Value for Solver 2
  model.Add(sum(assign_resource.flatten()) == obj)
  model.Maximize(cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return obj, solution, solver.ObjectiveValue()

In [17]:
''' Multiple-Objective Optimization Static Weighted : solve_static_weighted(intervals, num_resources) '''

def solve_static_weighted(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(10000*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [18]:
''' Multiple-Objective Optimization Dynamic Weighted : solve_dynamic_weighted(intervals, num_resources) '''

def solve_dynamic_weighted(intervals, num_resources):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # Grab Maximum Range of Intervals
  flattened = list(sum(intervals,()))
  task_weights = max(flattened) - min(flattened)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # 2D array for schedules | row represents a resource
  durations = np.zeros((num_resources, num_intervals))
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      durations[r][i] = interval[1]-interval[0]
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Weighted Sum for # of tasks + resource utilization time
  model.Maximize(task_weights*sum(assign_resource.flatten())+cp_model.LinearExpr.WeightedSum(assign_resource.flatten(), durations.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  num_scheduled = 0
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        num_scheduled += is_scheduled
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return num_scheduled, solution, solver.ObjectiveValue()

In [19]:
def cumulative_time(schedules): 
  total_time = 0
  for intervals in schedules: 
    for i in intervals: 
      total_time += i[1]-i[0]
  return total_time

def display_results(schedules): 
  for i, s in enumerate(schedules):
    print(f'Resource {i+1}:', s)

In [20]:
def check_testcase(testcase, approach1, approach2, output=True):
  output1 = approach1(*testcase)
  output2 = approach2(*testcase)
  sol1, sch1 = output1[0], output1[1]
  sol2, sch2 = output2[0], output2[1]

  # Ensure that optimal value matches 
  assert(sol1 == sol2)

  if output:
    print(f'Intervals: {str(testcase.intervals)}\nNumber of Resources: {testcase.num_resources}\n')
    print(f'Approach 1\tTotal Time: {cumulative_time(sch1)}')
    display_results(sch1)
    print('--------')
    print(f'Approach 2\tTotal Time: {cumulative_time(sch2)}')
    display_results(sch2)

testcase = namedtuple('testcase', ['intervals', 'num_resources'])

In [21]:
def get_runtime(approach, intervals, num_resources):
  start = timeit.default_timer()
  res = approach(intervals, num_resources)
  end = timeit.default_timer()
  return end-start, res

In [22]:
import random

def generate_intervals(num_intervals, start=0, stop=1440):
  intervals = []
  for i in range(num_intervals):
    x = random.randint(start, stop)
    y = random.randint(start, stop)
    if x == y:
      i -= 1
    elif x < y: 
      intervals += [(x, y)]
    else: 
      intervals += [(y,x)]
  return intervals


## Testing

In [21]:
intervals = [(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)]
num_resources = 2

# check_testcase(intervals, num_resources)
print(solve_schedule_opt(intervals, num_resources))
print(solve_static_weighted(intervals, num_resources))

intervals = [(1,3), (3,5), (5,7), (1,1000000000000), (2, 4), (4, 6), (6, 11), (2, 1000000000000)]
num_resources = 2

# check_testcase(intervals, num_resources)
print(solve_schedule_opt(intervals, num_resources))
print(solve_static_weighted(intervals, num_resources))
print(solve_dynamic_weighted(intervals, num_resources))

(5, [[(1, 3), (4, 11)], [(2, 5), (6, 8), (9, 12)]], 17.0)
(5, [[(1, 3), (4, 11)], [(2, 5), (6, 8), (9, 12)]], 50017.0)
(6, [[(1, 3), (3, 5), (5, 7)], [(2, 4), (4, 6), (6, 11)]], 15.0)
(2, [[(1, 1000000000000)], [(2, 1000000000000)]], 2000000019997.0)
(6, [[(1, 3), (3, 5), (5, 7)], [(2, 4), (4, 6), (6, 11)]], 6000000000009.0)


In [22]:
testcases = [
  # testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 1),
  # testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 2), 
  # testcase([(1,3), (4,11), (2,5), (6,8), (9,12), (7,10)], 3),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 1),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 2),
  testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 3),
  # testcase([(1,2), (3,10), (11,14), (1,4), (5,6), (6,13), (1,9), (9,12), (1,12)], 4),
  # testcase([(1,10), (1,15), (11,20), (16,20), (15,21)], 2),
  # testcase([(91, 683), (393, 952), (130, 242), (182, 450), (289, 564), (197, 198), (0, 694), (427, 435), (638, 1387), (619, 652), (24, 119), (1193, 1436), (455, 538), (203, 885), (683, 1339), (745, 1265), (395, 983), (541, 711), (963, 1238), (192, 831), (682, 879), (55, 409), (53, 117), (220, 251), (495, 887), (135, 265), (85, 324), (126, 400), (893, 1129), (1029, 1430), (1073, 1234), (102, 616), (193, 199), (258, 885), (72, 138), (6, 57), (88, 106), (94, 632), (562, 614), (328, 695), (213, 328), (538, 855), (297, 914), (124, 141), (84, 214), (986, 1278), (549, 1343), (346, 1309), (76, 93), (364, 732), (392, 410), (1027, 1308), (493, 586), (194, 1151), (58, 65), (220, 1289), (900, 1188), (399, 1213), (33, 39), (198, 382), (27, 238), (157, 437), (577, 598), (285, 531), (2, 20), (305, 1282), (1039, 1422), (1010, 1052), (0, 14), (531, 815), (263, 459), (80, 212), (15, 1274), (170, 615), (42, 283), (312, 509), (129, 164), (137, 1358), (495, 1186), (393, 401), (31, 555), (734, 740), (1109, 1263), (541, 1366), (31, 51), (420, 1143), (372, 1017), (254, 1097), (234, 1326), (67, 73), (53, 586), (42, 424), (445, 548), (394, 1069), (926, 1004), (7, 12), (264, 1182), (170, 1363), (189, 778), (262, 1022), (11, 55), (695, 1080), (575, 926), (748, 919), (142, 1376), (188, 733), (352, 508), (170, 274), (404, 461), (270, 351), (382, 1113)], 4)
]
for case in testcases: 
  check_testcase(case, solve_schedule_algo, solve_schedule_smt, False)
  print('Algorithmic Runtime: ', get_runtime(solve_schedule_algo, *case))
  print('SMT Synthesis Runtime: ', get_runtime(solve_schedule_smt, *case))
  print('\n.....\n')


Algorithmic Runtime:  (3.811300121014938e-05, (8, [[(1, 2), (3, 10), (11, 14)], [(1, 4), (5, 6), (6, 13)], [(1, 9), (9, 12)]]))
SMT Synthesis Runtime:  (0.06330162099766312, (8.0, [[(1, 9), (9, 12)], [(1, 4), (5, 6), (6, 13)], [(1, 2), (3, 10), (11, 14)]]))

.....



In [23]:
# testcase([(9,10:50), (9:30,11:45), (1, 2:50), (3-4:50), (11-12:50), (6, 6:50)])

check_testcase(testcase([(540, 650), (570, 705), (780, 890), (900, 1010), (660, 770), (1080, 1140)], 1), solve_schedule_algo, solve_schedule_smt)

Intervals: [(540, 650), (570, 705), (780, 890), (900, 1010), (660, 770), (1080, 1140)]
Number of Resources: 1

Approach 1	Total Time: 500
Resource 1: [(540, 650), (660, 770), (780, 890), (900, 1010), (1080, 1140)]
--------
Approach 2	Total Time: 500
Resource 1: [(540, 650), (780, 890), (900, 1010), (660, 770), (1080, 1140)]


In [24]:
intervals = [(542, 819), (308, 368), (456, 805), (238, 407), (1010, 1130), (990, 1009), (645, 1137), (19, 185), (264, 270), (354, 1064), (154, 301), (751, 1032), (255, 257), (91, 93), (639, 1385), (38, 88), (22, 828), (59, 414), (381, 621), (120, 230), (87, 499), (332, 1422), (61, 635), (61, 172), (661, 721), (226, 1307), (218, 305), (867, 1217), (663, 835)]
num_resources = 4

for i in range(10):
  num_resources = i
  print('i', i)
  print('Algorithmic Runtime: ', get_runtime(solve_schedule_algo, intervals, num_resources))
  print('SMT Synthesis Runtime: ', get_runtime(solve_schedule_smt, intervals, num_resources))
  print()


i 0
Algorithmic Runtime:  (4.299400097806938e-05, (0, []))
SMT Synthesis Runtime:  (0.0034259839994774666, (0.0, []))

i 1
Algorithmic Runtime:  (8.259899914264679e-05, (10, [[(38, 88), (91, 93), (120, 230), (255, 257), (264, 270), (308, 368), (381, 621), (661, 721), (990, 1009), (1010, 1130)]]))
SMT Synthesis Runtime:  (0.01636806300302851, (10.0, [[(308, 368), (1010, 1130), (990, 1009), (264, 270), (255, 257), (91, 93), (38, 88), (381, 621), (120, 230), (663, 835)]]))

i 2
Algorithmic Runtime:  (8.202800017897971e-05, (14, [[(38, 88), (91, 93), (120, 230), (255, 257), (264, 270), (456, 805), (990, 1009), (1010, 1130)], [(61, 172), (218, 305), (308, 368), (381, 621), (661, 721), (751, 1032)]]))
SMT Synthesis Runtime:  (0.5373686929997348, (14.0, [[(542, 819), (308, 368), (1010, 1130), (990, 1009), (264, 270), (255, 257), (91, 93), (38, 88), (120, 230)], [(381, 621), (61, 172), (218, 305), (867, 1217), (663, 835)]]))

i 3
Algorithmic Runtime:  (5.424299888545647e-05, (18, [[(38, 88), (

^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


SMT Synthesis Runtime:  (1.19587783300085, (20.0, [[(308, 368), (19, 185), (264, 270), (751, 1032), (255, 257), (661, 721)], [(381, 621), (61, 172), (218, 305), (867, 1217), (663, 835)], [(238, 407), (645, 1137), (91, 93), (38, 88), (120, 230)], [(456, 805), (1010, 1130), (990, 1009), (59, 414)]]))

i 5
Algorithmic Runtime:  (0.00010869499965338036, (22, [[(38, 88), (91, 93), (120, 230), (255, 257), (264, 270), (354, 1064)], [(61, 172), (238, 407), (542, 819), (867, 1217)], [(19, 185), (218, 305), (308, 368), (381, 621), (661, 721), (751, 1032)], [(154, 301), (663, 835), (990, 1009), (1010, 1130)], [(59, 414), (456, 805)]]))
SMT Synthesis Runtime:  (28.125999954998406, (22.0, [[(308, 368), (154, 301), (381, 621), (867, 1217), (663, 835)], [(238, 407), (751, 1032), (61, 172), (661, 721)], [(645, 1137), (19, 185), (218, 305)], [(264, 270), (255, 257), (91, 93), (38, 88), (120, 230), (332, 1422)], [(542, 819), (1010, 1130), (990, 1009), (59, 414)]]))

i 6
Algorithmic Runtime:  (0.00012140

^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


SMT Synthesis Runtime:  (23.387151989001723, (24.0, [[(308, 368), (381, 621), (61, 172), (218, 305), (867, 1217), (663, 835)], [(264, 270), (255, 257), (91, 93), (38, 88), (120, 230), (332, 1422)], [(238, 407), (19, 185), (751, 1032), (661, 721)], [(645, 1137), (61, 635)], [(639, 1385), (87, 499)], [(542, 819), (1010, 1130), (990, 1009), (59, 414)]]))

i 7
Algorithmic Runtime:  (7.704399831709452e-05, (26, [[(38, 88), (91, 93), (120, 230), (255, 257), (264, 270), (639, 1385)], [(61, 172), (238, 407), (645, 1137)], [(19, 185), (218, 305), (308, 368), (381, 621), (663, 835), (990, 1009), (1010, 1130)], [(154, 301), (354, 1064)], [(59, 414), (456, 805)], [(87, 499), (542, 819), (867, 1217)], [(61, 635), (661, 721), (751, 1032)]]))
SMT Synthesis Runtime:  (17.454325160000735, (26.0, [[(308, 368), (645, 1137), (381, 621), (61, 172), (218, 305)], [(61, 635), (867, 1217), (663, 835)], [(238, 407), (19, 185), (751, 1032), (661, 721)], [(639, 1385), (87, 499)], [(264, 270), (255, 257), (91, 93)

In [14]:
intervals = [(2,5), (14,19), (4,12), (13,15), (16,17), (18,20), (3,6), (7,10), (11,21), (1,22)]
num_resources = 2

alg_sol, alg_sch = solve_schedule_algo(intervals, num_resources)
display_results(alg_sch)

Resource 1: [(2, 5), (14, 19)]
Resource 2: [(3, 6), (7, 10), (13, 15), (16, 17), (18, 20)]


## Algorithm vs. SMT

In [15]:
''' CHANGE NUMBER OF RESOURCES '''
# intervals = [(542, 819), (308, 368), (456, 805), (238, 407), (1010, 1130), (990, 1009), (645, 1137), (19, 185), (264, 270), (354, 1064), (154, 301), (751, 1032), (255, 257), (91, 93), (639, 1385), (38, 88), (22, 828), (59, 414), (381, 621), (120, 230), (87, 499), (332, 1422), (61, 635), (61, 172), (661, 721), (226, 1307), (218, 305), (867, 1217), (663, 835), (158, 518)]
intervals = [(91, 683), (393, 952), (130, 242), (182, 450), (289, 564), (197, 198), (0, 694), (427, 435), (638, 1387), (619, 652), (24, 119), (1193, 1436), (455, 538), (203, 885), (683, 1339), (745, 1265), (395, 983), (541, 711), (963, 1238), (192, 831), (682, 879), (55, 409), (53, 117), (220, 251), (495, 887), (135, 265), (85, 324), (126, 400), (893, 1129), (1029, 1430), (1073, 1234), (102, 616), (193, 199), (258, 885), (72, 138), (6, 57), (88, 106), (94, 632), (562, 614), (328, 695), (213, 328), (538, 855), (297, 914), (124, 141), (84, 214), (986, 1278), (549, 1343), (346, 1309), (76, 93), (364, 732), (392, 410), (1027, 1308), (493, 586), (194, 1151), (58, 65), (220, 1289), (900, 1188), (399, 1213), (33, 39), (198, 382), (27, 238), (157, 437), (577, 598), (285, 531), (2, 20), (305, 1282), (1039, 1422), (1010, 1052), (0, 14), (531, 815), (263, 459), (80, 212), (15, 1274), (170, 615), (42, 283), (312, 509), (129, 164), (137, 1358), (495, 1186), (393, 401), (31, 555), (734, 740), (1109, 1263), (541, 1366), (31, 51), (420, 1143), (372, 1017), (254, 1097), (234, 1326), (67, 73), (53, 586), (42, 424), (445, 548), (394, 1069), (926, 1004), (7, 12), (264, 1182), (170, 1363), (189, 778), (262, 1022), (11, 55), (695, 1080), (575, 926), (748, 919), (142, 1376), (188, 733), (352, 508), (170, 274), (404, 461), (270, 351), (382, 1113)][:30]
# intervals = [(323, 1138), (759, 891), (81, 624), (473, 494), (1056, 1313), (635, 990), (442, 718), (120, 436), (1036, 1421), (267, 1315), (359, 833), (140, 593), (387, 455), (188, 1262), (835, 1248), (827, 927), (499, 616), (1065, 1305), (439, 1390), (658, 778), (89, 1327), (946, 1281), (426, 711), (155, 730), (64, 440), (933, 1188), (219, 801), (37, 727), (556, 1192), (241, 416), (953, 990), (87, 695), (615, 1330), (165, 1204), (603, 915), (784, 1033), (21, 362), (1057, 1436), (1000, 1174), (141, 731), (590, 1053), (1058, 1085), (412, 1072), (755, 1118), (439, 1127), (502, 655), (1253, 1344), (77, 678), (1365, 1403), (601, 1088), (742, 1354), (1027, 1401), (583, 1414), (182, 542), (410, 565), (78, 713), (696, 1431), (1118, 1187), (322, 601), (1150, 1151), (621, 870), (540, 908), (791, 845), (22, 167), (223, 497), (1245, 1272), (277, 795), (628, 1215), (664, 1017), (272, 629), (750, 1196), (541, 943), (689, 1375), (335, 1091), (454, 1322), (422, 611), (912, 1367), (724, 1129), (140, 531), (0, 527), (1143, 1159), (1034, 1154), (655, 676), (40, 226), (1204, 1254), (527, 919), (945, 1057), (608, 1166), (446, 994), (65, 524), (39, 1050), (158, 266), (650, 976), (222, 1054), (163, 1432), (187, 1068), (483, 1260), (388, 983), (862, 1436), (673, 789)]
# intervals = generate_intervals(20)
# print(intervals)
num_resources = 10

# print(intervals)
# alg_sol, alg_sch = solve_schedule_algo(intervals, num_resources)
# print(alg_sol)
# display_results(alg_sch)

print(len(intervals))
for i in range(1,11):
  print(i, get_runtime(solve_schedule_opt, intervals, i)[0], get_runtime(solve_dynamic_weighted, intervals, i)[0], sep='\t')



30
1	0.048843243999996844	0.01878389300000549
2	2.6611911469999967	0.7812618309999948
3	57.59259401999998	5.4279939369999965
4	521.181953236	34.901369124999974


^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


In [28]:
''' CHANGE NUMBER OF INTERVALS '''

for i in range(1,501):
  # print(i, end='\t')
  # for _ in range(3):
  intervals = generate_intervals(i)
  print(1000*get_runtime(solve_schedule_algo, intervals, 1)[0])
  # print()



0.03370300146343652
0.012425998647813685
0.008565000825910829
0.008107999747153372
0.00927900146052707
0.00924300002225209
0.01044100099534262
0.011172000085934997
0.012800001059076749
0.014975001249695197
0.017821999790612608
0.01833700116549153
0.013937000403529964
0.013132001186022535
0.013408000086201355
0.014280998584581539
0.014868999642203562
0.01580199932504911
0.016119000065373257
0.016816999050206505
0.017094998838729225
0.036579000152414665
0.07957600064401049
0.04141300087212585
0.0424939989898121
0.043571999412961304
0.04566000097838696
0.055470000006607734
0.08296099986182526
0.05518400030268822
0.16786300147941802
0.0767719993746141
0.05356399924494326
0.062225999499787576
0.05982100083201658
0.052981999033363536
0.05890700049349107
0.06098300036683213
0.05773900011263322
0.18745800116448663
0.06634400051552802
0.06997399941610638
0.07251399983942974
0.06283100083237514
0.07393100167973898
0.06630000098084565
0.06912400021974463
0.07029100015643053
0.0704060003045015
0.0

In [24]:
''' COMPARISON '''
intervals = [(91, 683), (393, 952), (130, 242), (182, 450), (289, 564), (197, 198), (0, 694), (427, 435), (638, 1387), (619, 652), (24, 119), (1193, 1436), (455, 538), (203, 885), (683, 1339), (745, 1265), (395, 983), (541, 711), (963, 1238), (192, 831), (682, 879), (55, 409), (53, 117), (220, 251), (495, 887), (135, 265), (85, 324), (126, 400), (893, 1129), (1029, 1430), (1073, 1234), (102, 616), (193, 199), (258, 885), (72, 138), (6, 57), (88, 106), (94, 632), (562, 614), (328, 695), (213, 328), (538, 855), (297, 914), (124, 141), (84, 214), (986, 1278), (549, 1343), (346, 1309), (76, 93), (364, 732), (392, 410), (1027, 1308), (493, 586), (194, 1151), (58, 65), (220, 1289), (900, 1188), (399, 1213), (33, 39), (198, 382), (27, 238), (157, 437), (577, 598), (285, 531), (2, 20), (305, 1282), (1039, 1422), (1010, 1052), (0, 14), (531, 815), (263, 459), (80, 212), (15, 1274), (170, 615), (42, 283), (312, 509), (129, 164), (137, 1358), (495, 1186), (393, 401), (31, 555), (734, 740), (1109, 1263), (541, 1366), (31, 51), (420, 1143), (372, 1017), (254, 1097), (234, 1326), (67, 73), (53, 586), (42, 424), (445, 548), (394, 1069), (926, 1004), (7, 12), (264, 1182), (170, 1363), (189, 778), (262, 1022), (11, 55), (695, 1080), (575, 926), (748, 919), (142, 1376), (188, 733), (352, 508), (170, 274), (404, 461), (270, 351), (382, 1113)][:100]

for i in range(105,201,5):
  intervals = generate_intervals(i)
  print(i, get_runtime(solve_schedule_smt, intervals, 1)[0],get_runtime(solve_schedule_opt, intervals, 1)[0],get_runtime(solve_dynamic_weighted, intervals, 1)[0],sep="\t")



105	0.6230480290000742	2.9390823510000246	0.6883707810000033
110	0.22607410799992067	0.6929058929999883	0.15426351599990085
115	0.38765676000002713	1.4178692350001256	0.3426248929999929
120	0.7801435839999158	4.441491402999873	0.6995160629999191
125	0.451859388999992	2.0237783180000406	0.6552453869999226
130	0.4422225769999386	2.444441459000018	0.5148522730000877
135	0.3258665900000324	1.103459267999824	0.5364080389999799
140	8.029075439000053	13.458035302000098	7.134770779999826
145	1.4797812960000556	4.501130730999876	1.5550906840001062
150	11.270465113	30.394249356000046	20.474479040999995
155	2.792066019999993	10.720048415000065	3.0854449959999783
160	2.413785141000062	9.064978376999989	3.9207732410000062
165	24.663713719000043	77.24577979500009	37.776758943999994
170	2.379193879000013	8.961289106000095	3.214130463000174
175	9.690205164999952	31.32085416400014	14.298511020999968
180	2.44948836399999	8.274368808999952	3.0636593609999636
185	8.016984202999993	11.885104201000104	8.549

## Extensions

In [ ]:
''' GISMP (Group Interval Scheduling Maximization Problem) '''

def solve_gismp(intervals, num_resources, groups):
  # Declare Model
  model = cp_model.CpModel()

  # Define Variables
  num_intervals = len(intervals)

  # 2D boolean table for interval-resource assignments
  # row represents resource | column represents interval
  assign_resource = np.empty((num_resources, num_intervals), dtype=cp_model.IntVar)
  for i in range(num_intervals):
    for r in range(num_resources):
      assign_resource[r][i] = model.NewBoolVar(f'assign_i{i+1}_r{r+1}')

  # groups = 
  for group in groups:
    for interval in group: 


  # 2D array for schedules | row represents a resource
  schedules = np.empty((num_resources, num_intervals), dtype=cp_model.IntervalVar)
  for i in range(num_intervals):
    interval = intervals[i]
    for r in range(num_resources):
      schedules[r][i] = model.NewOptionalIntervalVar(interval[0], interval[1]-interval[0], 
                        interval[1], assign_resource[r][i], f'interval_i{i+1}_r{r+1}')

  # No overlapping intervals
  for r in range(num_resources):
    model.AddNoOverlap(schedules[r])

  # Each task scheduled at most once
  for i in range(num_intervals): 
    model.AddAtMostOne(assign_resource[:,i])

  # Maximize Length of Schedule
  model.Maximize(sum(assign_resource.flatten()))

  # Invoke Solver
  solver = cp_model.CpSolver()
  status = solver.Solve(model)

  # Reformat Solution 
  solution = []
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for r in range(num_resources):
      resource_schedule = []
      for i in range(num_intervals):
        is_scheduled = solver.Value(assign_resource[r][i])
        if is_scheduled: resource_schedule.append(intervals[i])
      solution.append(resource_schedule)
  else: 
    print(f'Solver Failed with Error Code: {solver.StatusName()}')

  return solver.ObjectiveValue(), solution